In [1]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import numpy as np
import pandas as pd

In [2]:
distributions_df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 3 Files\Questions\distributions.csv")
cost_matrix_df = pd.read_csv(r"C:\Users\johns\OneDrive\Desktop\MBAN Semester 3\OMIS 6000 - Models & Applications in Operational Research\Assignment 3 Files\Questions\cost_matrix.csv")

In [3]:
selling_price = 49.99
ordering_cost = 24.44
n_stores = len(distributions_df)
n_scenarios = 100
n_trials = 50
opportunity_cost_per_unit = selling_price - ordering_cost

In [4]:
def generate_demand_scenarios(n_trials, n_scenarios, distributions_df):
    demand_scenarios = np.zeros((n_trials, n_scenarios, n_stores))
    for trial in range(n_trials):
        for scenario in range(n_scenarios):
            for store in range(n_stores):
                n = distributions_df.loc[store, 'n']
                p = distributions_df.loc[store, 'p']
                demand_scenarios[trial, scenario, store] = np.random.binomial(n, p)
    return demand_scenarios

In [5]:
def solve_optimization_model_no_transshipment(demand_scenario):
    model = gp.Model("OptimalOrderQuantityNoTransshipment")
    model.setParam('OutputFlag', 0)

    # Decision variables
    y = model.addVars(n_stores, vtype=GRB.CONTINUOUS, name="order_qty", lb=0)
    unmet_demand = model.addVars(n_stores, vtype=GRB.CONTINUOUS, name="unmet_demand", lb=0)

    # Objective function
    model.setObjective(
        quicksum(ordering_cost * y[i] for i in range(n_stores)) + 
        quicksum(unmet_demand[i] * opportunity_cost_per_unit for i in range(n_stores)), 
        GRB.MINIMIZE
    )

    # Constraints
    for i in range(n_stores):
        model.addConstr(y[i] + unmet_demand[i] >= demand_scenario[i], name=f"demand_met_{i}")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        return [y[i].X for i in range(n_stores)], model.ObjVal
    else:
        return None, None

In [6]:
def solve_optimization_model_with_transshipment(demand_scenario):
    model = gp.Model("OptimalOrderQuantityWithTransshipment")
    model.setParam('OutputFlag', 1)  # Enable output for debugging

    # Decision variables
    y = model.addVars(n_stores, vtype=GRB.CONTINUOUS, name="order_qty")
    x = model.addVars(n_stores, n_stores, vtype=GRB.CONTINUOUS, name="transshipment")
    unmet_demand = model.addVars(n_stores, vtype=GRB.CONTINUOUS, name="unmet_demand", lb=0)
    z = model.addVars(n_stores, n_stores, vtype=GRB.BINARY, name="transshipment_indicator")

    # Objective function
    model.setObjective(
        quicksum(ordering_cost * y[i] for i in range(n_stores)) + 
        quicksum(cost_matrix_df.iloc[i, j] * x[i, j] for i in range(n_stores) for j in range(n_stores) if i != j) + 
        quicksum(opportunity_cost_per_unit * unmet_demand[i] for i in range(n_stores)),
        GRB.MINIMIZE
    )

    # Constraints
    M = max(cost_matrix_df.max().max(), np.max(demand_scenario)) * 1.1  # Set M to be slightly larger than the maximum possible transshipment quantity
    
    for i in range(n_stores):
        model.addConstr(
            y[i] + quicksum(x[j,i] for j in range(n_stores) if j != i) -
            quicksum(x[i,j] for j in range(n_stores) if i != j) + unmet_demand[i] == demand_scenario[i],
            "DemandFulfillment_{}".format(i)
        )

    for i in range(n_stores):
        for j in range(n_stores):
            if i != j:
                model.addConstr(
                    x[i, j] <= M * z[i, j],
                    "TransshipmentDecision_{}_{}".format(i, j)
                )

    model.optimize()

    if model.status == GRB.OPTIMAL:
        return [y[i].X for i in range(n_stores)], model.ObjVal
    else:
        return None, None

In [7]:
demand_scenarios = generate_demand_scenarios(n_trials, n_scenarios, distributions_df)

In [8]:
# No transshipment scenario
average_order_quantities_no_transshipment = np.zeros(n_stores)
average_total_cost_no_transshipment = 0

In [9]:
for trial in range(n_trials):
    for scenario in range(n_scenarios):
        demand_scenario = demand_scenarios[trial, scenario, :]
        optimal_order_quantities, total_cost = solve_optimization_model_no_transshipment(demand_scenario)
        if optimal_order_quantities is not None:
            average_order_quantities_no_transshipment += np.array(optimal_order_quantities)
            average_total_cost_no_transshipment += total_cost

Restricted license - for non-production use only - expires 2025-11-24


In [10]:
average_order_quantities_no_transshipment /= (n_trials * n_scenarios)
average_total_cost_no_transshipment /= (n_trials * n_scenarios)

print(f"Average Order Quantities (No Transshipment): {average_order_quantities_no_transshipment}")
print(f"Average Total Cost (No Transshipment): {average_total_cost_no_transshipment}")

Average Order Quantities (No Transshipment): [ 89.9416  51.8344  99.054   66.7476  53.4118  61.6404 114.6898 110.0004
  99.6158  98.2774  82.3936  95.2226 120.1444 121.8576 115.3974]
Average Total Cost (No Transshipment): 33732.791872


In [11]:
# With transshipment scenario
average_order_quantities_with_transshipment = np.zeros(n_stores)
average_total_cost_with_transshipment = 0

In [12]:
for trial in range(n_trials):
    for scenario in range(n_scenarios):
        demand_scenario = demand_scenarios[trial, scenario, :]
        optimal_order_quantities, total_cost = solve_optimization_model_with_transshipment(demand_scenario)
        if optimal_order_quantities is not None:
            average_order_quantities_with_transshipment += np.array(optimal_order_quantities)
            average_total_cost_with_transshipment += total_cost

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 225 rows, 480 columns and 870 nonzeros
Model fingerprint: 0xceaf1afa
Variable types: 255 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
Found heuristic solution: objective 34191.560000
Presolve removed 210 rows and 255 columns
Presolve time: 0.00s
Presolved: 15 rows, 225 columns, 435 nonzeros
Variable types: 225 continuous, 0 integer (0 binary)

Root relaxation: cutoff, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0

In [ ]:
average_order_quantities_with_transshipment /= (n_trials * n_scenarios)
average_total_cost_with_transshipment /= (n_trials * n_scenarios)

print(f"Average Order Quantities (With Transshipment): {average_order_quantities_with_transshipment}")
print(f"Average Total Cost (With Transshipment): {average_total_cost_with_transshipment}")

Average Order Quantities (With Transshipment): [ 89.9744  51.821   99.1168  66.769   53.4412  61.7918 114.6776 110.035
  99.6034  98.2978  82.3864  95.2104 120.1126 121.776  115.3376]
Average Total Cost (With Transshipment): 33735.77844000001


In [ ]:
n_scenarios = 500
n_trials = 1

In [ ]:
demand_scenarios = generate_demand_scenarios(n_trials, n_scenarios, distributions_df)[0]
average_demand = np.mean(demand_scenarios, axis=0)
_, deterministic_total_cost = solve_optimization_model_with_transshipment(average_demand)

In [ ]:
total_cost_stochastic = 0

In [ ]:
for scenario in demand_scenarios:
    optimal_order_quantities, scenario_cost = solve_optimization_model_with_transshipment(scenario)
    if optimal_order_quantities is not None:
        total_cost_stochastic += scenario_cost

In [ ]:
average_total_cost_stochastic = total_cost_stochastic / n_scenarios
VSS = average_total_cost_stochastic - deterministic_total_cost

In [ ]:
print(f"EEV (Expected Ex-post Value): {average_total_cost_stochastic}")
print(f"VSS (Value of the Stochastic Solution): {VSS}")

EEV (Expected Ex-post Value): 33728.666400000024
VSS (Value of the Stochastic Solution): 1.4551915228366852e-11
